In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Multiple_sclerosis/GSE193442'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptional profiling of human KIR+ CD8 T cells"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: PBMC'], 1: ['cell type: KIR+ CD8 T']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check if the dataset contains gene expression data
if 'Transcriptional profiling' in '!Series_title' and 'KIR+ CD8 T cells' in '!Series_summary':
    is_gene_available = True

# Define functions to convert values for each variable
def convert_trait(value):
    value = value.split(':')[-1].strip().lower()
    if value == 'multiple_sclerosis':
        return 1
    elif value == 'control':
        return 0
    else:
        return None

def convert_age(value):
    try:
        age_value = int(value.split(':')[-1].strip())
        return age_value
    except ValueError:
        return None

def convert_gender(value):
    gender_value = value.split(':')[-1].strip().lower()
    if gender_value == 'female':
        return 0
    elif gender_value == 'male':
        return 1
    else:
        return None

# Identify keys from the sample characteristics dictionary
sample_characteristics = {
    0: ['tissue: PBMC'], 
    1: ['cell type: KIR+ CD8 T']
}

for key, values in sample_characteristics.items():
    for value in values:
        if 'multiple_sclerosis' in value.lower():
            trait_row = key
        if 'age' in value.lower():
            age_row = key
        if 'gender' in value.lower():
            gender_row = key

# Save cohort information
save_cohort_info('GSE193442', './preprocessed/Multiple_sclerosis/cohort_info.json', is_gene_available, trait_row is not None)

# If clinical data is available, extract clinical features
if trait_row is not None:
    selected_clinical_data = geo_select_clinical_features(clinical_data, 'Multiple_sclerosis', trait_row, convert_trait, age_row, convert_age, gender_row, convert_gender)
    csv_path = './preprocessed/Multiple_sclerosis/trait_data/GSE193442.csv'
    selected_clinical_data.to_csv(csv_path)
    print(preview_df(selected_clinical_data))


A new JSON file was created at: ./preprocessed/Multiple_sclerosis/cohort_info.json
